<a href="https://colab.research.google.com/github/kosirobwada/DeepLearningFromZero2/blob/main/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第3章　word2vec


# カウントベースの手法の問題点

・現実的には、コーパスで扱う語彙数は非常に巨大になる。

・すると、巨大な行列を作らなければならず、SVDを行うのが現実的でない。SVDは、O(N^3)

# 推論ベースの手法

・ニューラルネットワークを用いる場合は、ミニバッチ学習をする。

・ミニバッチ学習をするとは、一度に少量のミニバッチの学習サンプルを見ながら重み更新を繰り返す。

・推論ベースの手法では、学習データの一部を使って逐次的に計算する。

・これにより、計算量が膨大で処理が難しい場合でも、ニューラルネットワークでは処理を小分けにして実行できる。

# one-hot 表現

・one-hot表現とは、ベクトルの要素の中でひとつだけが1で残りが0のベクトル。

In [1]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
h = np.dot(c,W)
print(h)

[[-0.18300872  1.52038272  0.96223897]]


In [5]:
class MatMul:
  def __init__(self,W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.x = None

  def forward(self, x):
    W, = self.params
    out = np.dot(x, W)
    self.x = x
    return out

  def backward(self, dout):
    W, = self.params
    dx = np.dot(dout, W.T)
    dW = np.dot(self.x.T, dout)
    self.grads[0][...] = dW
    return dx

c = np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-1.14192312  0.65870819  1.16475768]]


# CBOW(Continuous Bag-Of-Words)

・CBOWモデルは、コンテクストから、ターゲットを推測することを目的としたニューラルネットワークのモデル。

・ターゲットは、中央の単語、その周辺の単語がコンテクスト

・CBOWモデルをできるだけ正確な推測ができるように訓練することで、私たちは単語の分散表現を獲得することができる。

・CBOWモデルへの入力は、単語のリスト。それをone-hot表現に変換することで、CBOWモデルが処理できるように調整する。

・入力層としてN個の単語を扱うのであれば、N個の入力層が存在。

In [7]:
c0 = np.array([[1,0,0,0,0,0,0]])
c1 = np.array([[0,0,1,0,0,0,0]])

W_in = np.random.randn(7,3)
W_out = np.random.randn(3,7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[-0.64544172 -0.08348532  0.16479746 -2.22382516  0.25740015  0.82392375
   0.73392399]]


# CBOWモデルの学習

・これまで見てきたCBOWモデルは、出力層ごとに各単語のスコアを出力してきた。

・このスコアに対して、Softmax関数を適用することで、確率を得ることができる。

・この確率は、コンテクストが与えられたときに、その中央にどの単語が出現するのかを表す。